In [3]:
# Vamos a ver qué columnas tienen las tablas clave para entender cómo unirlas
print("--- 🛒 TABLA DE ÓRDENES (orders) ---")
print(olist_data['orders'].columns.tolist())
print("\n--- 📦 TABLA DE ITEMS (order_items) ---")
print(olist_data['order_items'].columns.tolist())
print("\n--- 👤 TABLA DE CLIENTES (customers) ---")
print(olist_data['customers'].columns.tolist())

--- 🛒 TABLA DE ÓRDENES (orders) ---
['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']

--- 📦 TABLA DE ITEMS (order_items) ---
['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']

--- 👤 TABLA DE CLIENTES (customers) ---
['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']


In [2]:
import pandas as pd
import os

# --- CORRECCIÓN DE RUTA ---
# Verificamos dónde estamos parados
current_path = os.getcwd()
print(f"📍 Estás ejecutando desde: {current_path}")

# Si la ruta termina en 'notebooks', subimos un nivel para ir a la raíz del proyecto
if current_path.endswith('notebooks') or current_path.endswith('notebooks/'):
    project_root = os.path.dirname(current_path)
else:
    project_root = current_path

print(f"🏠 Raíz del proyecto detectada: {project_root}")

# Definir la ruta correcta a data/raw
raw_data_path = os.path.join(project_root, 'data', 'raw')
print(f"📂 Buscando datos en: {raw_data_path}\n")

# --- CARGA DE DATOS ---
if os.path.exists(raw_data_path):
    files = [f for f in os.listdir(raw_data_path) if f.endswith('.csv')]
    
    olist_data = {}
    print(f"✅ ¡Ruta encontrada! Cargando {len(files)} archivos...\n")
    
    for file in files:
        # Limpiar nombre
        table_name = file.replace('olist_', '').replace('_dataset.csv', '').replace('product_category_name_translation.csv', 'category_translation')
        
        # Cargar
        file_path = os.path.join(raw_data_path, file)
        olist_data[table_name] = pd.read_csv(file_path)
        
        print(f"📦 Tabla: {table_name:<25} | Filas: {len(olist_data[table_name]):,d}")
else:
    print("❌ Error crítico: Aún no encuentro la carpeta. Verifica que 'data' y 'notebooks' estén al mismo nivel.")

📍 Estás ejecutando desde: c:\Users\Eduar\Documents\churn-project\notebooks
🏠 Raíz del proyecto detectada: c:\Users\Eduar\Documents\churn-project
📂 Buscando datos en: c:\Users\Eduar\Documents\churn-project\data\raw

✅ ¡Ruta encontrada! Cargando 9 archivos...

📦 Tabla: customers                 | Filas: 99,441
📦 Tabla: geolocation               | Filas: 1,000,163
📦 Tabla: orders                    | Filas: 99,441
📦 Tabla: order_items               | Filas: 112,650
📦 Tabla: order_payments            | Filas: 103,886
📦 Tabla: order_reviews             | Filas: 99,224
📦 Tabla: products                  | Filas: 32,951
📦 Tabla: sellers                   | Filas: 3,095
📦 Tabla: category_translation      | Filas: 71


# 1. Definición del Problema de Churn (Contexto Olist)

## 1.1 Objetivo del Negocio

El objetivo principal es reducir la tasa de abandono de **clientes (compradores)**, enfocando los esfuerzos de retención solo en aquellos con alto potencial de *churn* que, si se retienen, maximizarán el Retorno de la Inversión (ROI).

## 1.2 Definición Operacional de Churn (Basada en Olist)

Dado que Olist es una plataforma de *e-commerce* con compras no periódicas, definiremos el *churn* de la siguiente manera:

* **Unidad de Predicción:** Un **Cliente (customer)**.
* **Comportamiento de Referencia (Baseline):** La actividad normal de un cliente es realizar una compra.
* **Horizonte de Abandono (T):** Un cliente es considerado **"Churn"** si **no ha realizado una nueva compra en `T = 90 días`** después de su última transacción.
* **Etiqueta (Target):**
    * **1 (Churn):** Si el cliente no compró en los 90 días posteriores a nuestra "Fecha de Corte".
    * **0 (No Churn / Activo):** Si el cliente sí compró dentro de los 90 días.

## 1.3 Métrica de Negocio (KPI)

La métrica principal no será la precisión del modelo, sino el impacto financiero:

* **Métrica Clave:** **Uplift de Retención**. Se medirá como el porcentaje de clientes predichos como Churn (grupo de Tratamiento) que, gracias a una intervención de retención, vuelven a comprar, en comparación con un grupo de Control que no recibió intervención.
* **Fórmula:** Tasa de Retención (Tratamiento) - Tasa de Retención (Control) = **Uplift**